# Strategy Serialization with BoFire

## Imports

In [1]:
import json
from pydantic import parse_obj_as


from bofire.data_models.domain.api import Domain, Outputs
from bofire.benchmarks.single import Himmelblau
from bofire.benchmarks.multi import DTLZ2
from bofire.data_models.strategies.api import SoboStrategy as SoboStrategyDataModel
from bofire.data_models.strategies.api import QnehviStrategy as QnehviStrategyDataModel
from bofire.data_models.strategies.api import RandomStrategy as RandomStrategyDataModel
from bofire.data_models.strategies.api import AnyStrategy
from bofire.data_models.acquisition_functions.api import qNEI
import bofire.strategies.api as stategies
from bofire.data_models.surrogates.api import BotorchSurrogates, SingleTaskGPSurrogate
from bofire.data_models.kernels.api import ScaleKernel, RBFKernel
from bofire.surrogates.trainable import TrainableSurrogate
from bofire.surrogates.diagnostics import CvResults2CrossValidationValues




/opt/homebrew/Caskroom/miniforge/base/envs/bofire/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Single Objective Problem Setup

In [2]:
benchmark = Himmelblau()
samples = benchmark.domain.inputs.sample(n=10)

# this is the training data
experiments = benchmark.f(samples, return_complete=True)

# this are the pending candidates
pending_candidates = benchmark.domain.inputs.sample(2)



## Random Strategy

The random strategy and other strategies that just inherit from `Strategy` and not `PredictiveStrategy` are special as they do not need defined output features in the domain and they do not need a call to `tell` before the `ask`. Furthermore they online provide input features in the candidates and no predictions for output features.

In [3]:
# setup the data model
domain = Domain(inputs=benchmark.domain.inputs)
strategy_data = RandomStrategyDataModel(domain=domain)

# we generate the json spec
jspec = strategy_data.json()

jspec

'{"type": "RandomStrategy", "domain": {"type": "Domain", "inputs": {"type": "Inputs", "features": [{"type": "ContinuousInput", "key": "x_1", "unit": null, "bounds": [-6.0, 6.0], "stepsize": null}, {"type": "ContinuousInput", "key": "x_2", "unit": null, "bounds": [-6.0, 6.0], "stepsize": null}]}, "outputs": {"type": "Outputs", "features": []}, "constraints": {"type": "Constraints", "constraints": []}}, "seed": null}'

In [4]:
# load it
strategy_data = parse_obj_as(AnyStrategy, json.loads(jspec))

# map it
strategy = stategies.map(strategy_data)

# ask it
df_candidates = strategy.ask(candidate_count=5)

# transform to spec
candidates = strategy.to_candidates(df_candidates)

candidates

[Candidate(inputValues={'x_1': InputValue(value=5.915408998829129), 'x_2': InputValue(value=3.8295447360002854)}, outputValues=None),
 Candidate(inputValues={'x_1': InputValue(value=2.16334840965789), 'x_2': InputValue(value=0.4058598910946323)}, outputValues=None),
 Candidate(inputValues={'x_1': InputValue(value=4.039471773229058), 'x_2': InputValue(value=5.531466507514837)}, outputValues=None),
 Candidate(inputValues={'x_1': InputValue(value=-1.783237287627132), 'x_2': InputValue(value=3.483195651684438)}, outputValues=None),
 Candidate(inputValues={'x_1': InputValue(value=1.89108688339689), 'x_2': InputValue(value=4.218035162071792)}, outputValues=None)]

## SOBO Strategy

Setup the strategies data model.

In [5]:
# setup the data model
strategy_data = SoboStrategyDataModel(domain=benchmark.domain, acquisition_function=qNEI())

# we generate the json spec
jspec = strategy_data.json()

jspec

'{"type": "SoboStrategy", "domain": {"type": "Domain", "inputs": {"type": "Inputs", "features": [{"type": "ContinuousInput", "key": "x_1", "unit": null, "bounds": [-6.0, 6.0], "stepsize": null}, {"type": "ContinuousInput", "key": "x_2", "unit": null, "bounds": [-6.0, 6.0], "stepsize": null}]}, "outputs": {"type": "Outputs", "features": [{"type": "ContinuousOutput", "key": "y", "unit": null, "objective": {"type": "MinimizeObjective", "w": 1.0, "bounds": [0, 1]}}]}, "constraints": {"type": "Constraints", "constraints": []}}, "seed": null, "num_sobol_samples": 512, "num_restarts": 8, "num_raw_samples": 1024, "descriptor_method": "EXHAUSTIVE", "categorical_method": "EXHAUSTIVE", "discrete_method": "EXHAUSTIVE", "surrogate_specs": {"surrogates": [{"hyperconfig": {"type": "SingleTaskGPHyperconfig", "hyperstrategy": "FactorialStrategy", "inputs": {"type": "Inputs", "features": [{"type": "CategoricalInput", "key": "kernel", "categories": ["rbf", "matern_1.5", "matern_2.5"], "allowed": [true, t

As SOBO is a predictive strategy, training data has to be provided before candidated can be requested. 

In [6]:
# load it
strategy_data = parse_obj_as(AnyStrategy, json.loads(jspec))

# map it
strategy = stategies.map(strategy_data)

# tell it the pending candidates if present
if pending_candidates is not None:
    strategy.add_candidates(pending_candidates)

# tell it
strategy.tell(experiments=experiments)

# ask it
df_candidates = strategy.ask(candidate_count=2)

# transform to spec
candidates = strategy.to_candidates(df_candidates)

candidates

[Candidate(inputValues={'x_1': InputValue(value=2.761549413905569), 'x_2': InputValue(value=0.6996718005451535)}, outputValues={'y': OutputValue(predictedValue=-299.9235749427238, standardDeviation=482.01269786145133, objective=299.9235749427238)}),
 Candidate(inputValues={'x_1': InputValue(value=1.43396385228208), 'x_2': InputValue(value=2.686897099761646)}, outputValues={'y': OutputValue(predictedValue=-321.26193726367933, standardDeviation=410.6722034209191, objective=321.26193726367933)})]

We can also save the trained models of the strategy, for more info look at the `model_serial.ipynb` notebook. It could be that the `dumps` command fails here. But this is already fixed in the main branch of the `linear_operator` package, and if not yet, it should be available in main soon.

In [7]:
jsurrogate_spec = strategy_data.surrogate_specs.surrogates[0].json()
dump = strategy.surrogates.surrogates[0].dumps()

## MOBO Strategy

As example for a multiobjective strategy we are using here the Qnehvi stratey. Related strategies would be Qparego, MultiplicativeSobo etc. To use it, we have to first generate a multiobjective domain.

In [8]:
benchmark = DTLZ2(dim=6)
samples = benchmark.domain.inputs.sample(n=20)
experiments = benchmark.f(samples, return_complete=True)
pending_candidates = benchmark.domain.inputs.sample(2)


Now the strategy spec is setup. Note that we can define there exactly which model to use.

In [9]:
# setup the data model
strategy_data = QnehviStrategyDataModel(
    domain=benchmark.domain,
    surrogate_specs=BotorchSurrogates(
        surrogates=[
            SingleTaskGPSurrogate(
                inputs=benchmark.domain.inputs,
                outputs=Outputs(features=[benchmark.domain.outputs[0]]),
                kernel=ScaleKernel(base_kernel=RBFKernel(ard=False))
            )
        ]
    )
)

# we generate the json spec
jspec = strategy_data.json()

jspec

'{"type": "QnehviStrategy", "domain": {"type": "Domain", "inputs": {"type": "Inputs", "features": [{"type": "ContinuousInput", "key": "x_0", "unit": null, "bounds": [0.0, 1.0], "stepsize": null}, {"type": "ContinuousInput", "key": "x_1", "unit": null, "bounds": [0.0, 1.0], "stepsize": null}, {"type": "ContinuousInput", "key": "x_2", "unit": null, "bounds": [0.0, 1.0], "stepsize": null}, {"type": "ContinuousInput", "key": "x_3", "unit": null, "bounds": [0.0, 1.0], "stepsize": null}, {"type": "ContinuousInput", "key": "x_4", "unit": null, "bounds": [0.0, 1.0], "stepsize": null}, {"type": "ContinuousInput", "key": "x_5", "unit": null, "bounds": [0.0, 1.0], "stepsize": null}]}, "outputs": {"type": "Outputs", "features": [{"type": "ContinuousOutput", "key": "f_0", "unit": null, "objective": {"type": "MinimizeObjective", "w": 1.0, "bounds": [0, 1]}}, {"type": "ContinuousOutput", "key": "f_1", "unit": null, "objective": {"type": "MinimizeObjective", "w": 1.0, "bounds": [0, 1]}}]}, "constraint

Generate the candidates.

In [10]:
# load it
strategy_data = parse_obj_as(AnyStrategy, json.loads(jspec))

# map it
strategy = stategies.map(strategy_data)

# tell it the pending candidates if available
if pending_candidates is not None:
    strategy.add_candidates(pending_candidates)

# tell it
strategy.tell(experiments=experiments)

# ask it
df_candidates = strategy.ask(candidate_count=1)

# transform to spec
candidates = strategy.to_candidates(df_candidates)

candidates

[Candidate(inputValues={'x_0': InputValue(value=1.0), 'x_1': InputValue(value=0.41791626265227944), 'x_2': InputValue(value=0.24511743692538823), 'x_3': InputValue(value=0.8373427510871627), 'x_4': InputValue(value=0.5999227288919193), 'x_5': InputValue(value=0.2274593719111656)}, outputValues={'f_0': OutputValue(predictedValue=0.16971265416997772, standardDeviation=0.14554309452460695, objective=-0.16971265416997772), 'f_1': OutputValue(predictedValue=1.0293363218546256, standardDeviation=0.2456146332167214, objective=-1.0293363218546256)})]

To fill the model info section accordingly, the following snippet has to be executed for every surrogate, incldung saving the actual models.

In [11]:
from pydantic import BaseModel
from typing import Literal

class TestMethod(BaseModel):
    type: str

class CrossValidation(TestMethod):
    type: Literal["CrossValidation"] = "CrossValidation"
    foldCount: int


for i in range(len(strategy_data.surrogate_specs.surrogates)):
    surrogate_data = strategy.surrogate_specs.surrogates[i]
    surrogate = strategy.surrogates.surrogates[i]
    # get the spec
    jsurrogate_spec = surrogate_data.json()
    # get the dump
    dump = surrogate.dumps()
    # do the cross validation, only if we have a trainable model under the hood
    if isinstance(surrogate,TrainableSurrogate):
        cv_train, cv_test, _ = surrogate.cross_validate(strategy.experiments, folds=5)
        # transform the bofire objects to the backend objects
        testMethod = CrossValidation(foldCount=5)
        cvResultsTrain = CvResults2CrossValidationValues(cv_train)
        cvResultsTest = CvResults2CrossValidationValues(cv_test)
        metricsTrain = {surrogate.outputs[0].key: cv_train.get_metrics(combine_folds=False).describe().loc["mean"].to_dict()}
        metricsTest = {surrogate.outputs[0].key: cv_test.get_metrics(combine_folds=True).describe().loc["mean"].to_dict()}
        # save to backend
        # - jsurrogate_spec
        # - dump
        # - testMethod
        # - cvResultsTrain
        # - cvResultsTest
        # - metricsTrain
        # - metricsTest